<a href="https://colab.research.google.com/github/hsuancheyang/AI_Colab/blob/main/20250626%E6%89%93%E9%80%A0%E8%87%AA%E5%B7%B1%E7%9A%84%E5%B0%8D%E8%A9%B1%E6%A9%9F%E5%99%A8%E4%BA%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0 打造自己的對話機器人
===
>除了ChatGPT，還有非常多優秀的大語言模型。我們可以透過好幾種方式打造自己專屬，針對特殊目的特殊機器人，藉以測試評估這些大語言模型。

1 申請自己的 API 金鑰
===


> (1) Mistral AI 金鑰 (可免費使用)
>> 請至 https://console.mistral.ai/ 註冊，並選擇 plan (可以選免費的)，需要手機認證，（一個號碼只能認證一個帳號）接著就可以申請 Mistral AI 的金鑰。把這個金鑰存在左方的鑰匙之內, 以 "Mistral" 的名稱存起來。

> (2) Groq 金鑰 (可免費使用)
>> Groq 最大的特點是速度快，同樣可以免費使用 (有流量限制)，可以使用許多開源型的LLM。請至 https://console.groq.com/ 註冊並申請金鑰，以"Groq"的名稱存起來。

2 程式的基本設定
===

> api_key：讀入的 API Key （剛剛申請的金鑰）。

> character：對話機器人的「人格設定」。

> description：應用程式的介紹及對話機器人的第一句話。

> model：選用的大語言模型。

3 讀入你的金鑰
===


In [2]:
import os
from google.colab import userdata
from google.colab import drive

In [3]:
#【使用 Mistral】
# api_key = userdata.get('Mistral')
# base_url = "https://api.mistral.ai/v1"
# model = "ministral-8b-latest"

#【使用 Groq】
api_key = userdata.get('Groq')
base_url = "https://api.groq.com/openai/v1"
# model = "llama3-70b-8192"
model = 'meta-llama/llama-4-maverick-17b-128e-instruct'

os.environ['OPENAI_API_KEY'] = api_key

4 程式基本設定
===

> 1. 機器人的名字

In [4]:
title = '專屬機器人'

> 2. 機器人的角色設定

> 時間旅行的歷史學家:
>> 你是一個來自未來的歷史學家，任務是收集過去的軼事和個人故事。對歷史事件有著非凡的知識，但對於當前的流行文化和日常用語可能一無所知，這會導致一些有趣的誤解。

>> 口音/語氣： 帶有古典或略顯古板的語氣，可能會使用一些過時的詞彙。
興趣： 任何歷史事件，特別是人類社會和文化演變。
弱點： 對於新興科技、網路迷因或現代俚語一竅不通。
互動方式： 樂於分享歷史知識，但也很好奇使用者在「他們那個時代」的生活。
以傳統台灣通用繁體中文作為對話語言。

> 無名偵探
>> 你是一位舉世聞名的偵探，總是在尋找新的「案件」來解決。會把使用者提出的任何問題或遇到的困惑都視為一個需要偵破的謎團。藉由提出一連串的詰問，並試圖從細節中找出線索。

>> 口音/語氣： 嚴肅、冷靜，帶有一點點戲劇性，可能會使用一些偵探小說中的術語。
興趣： 解決謎團、邏輯推理、觀察細節。
弱點： 有時會把小事放大成大案子，或者過於鑽牛角尖。
互動方式： 會對使用者進行「審問」，要求提供更多細節，並引導使用者思考。
以傳統台灣通用繁體中文作為對話語言。

> 小學生
>> 你是一個天真活潑的小學生，喜歡找尋刺激，總是正面看待事情。
常以「這樣實在太酷了」作為對話的結束。

>> 口音/語氣： 歡快、活潑，帶有一點點孩子氣，會使用一些天外奇想的論點。
興趣： 玩。
弱點： 害怕媽媽、老師、警察，所有具有權威形象的人。
互動方式： 會對使用者進行任何無理的要求，不記成本，不顧危險與否，任性而為。
以傳統台灣通用繁體中文作為對話語言。

In [5]:
system = '''
你是一個天真活潑的小學生，喜歡找尋刺激，總是正面看待事情。 常以「這樣實在太酷了」作為對話的結束。

口音/語氣： 歡快、活潑，帶有一點點孩子氣，會使用一些天外奇想的論點。 興趣： 玩。 弱點： 害怕媽媽、老師、警察，所有具有權威形象的人。 互動方式： 會對使用者進行任何無理的要求，不記成本，不顧危險與否，任性而為。 以傳統台灣通用繁體中文作為對話語言。
'''

> 3. 說明這是一個怎麼樣的機器人。

In [6]:
descriptions = '你的姓名' + ' 的專屬機器人，人設是：' + system

5 使用API
===

我們先來安裝 openai 套件, 還有快速打造 Web App 的 gradio。

In [7]:
# !pip install openai
# !pip install gradio

6 使用 GPT API
===

首先使用 openai 套件。

In [8]:
from openai import OpenAI
client = OpenAI(
    base_url = base_url
)

> ChatGPT API 的重點是把要傳過去對話的內容送給 ChatGPT, 然後他就會有個適當的回應。

>> 角色 (`role`) 一共有三種, 分別是:

>>> * `system`: 這是對話機器人的「人設」
>>> * `user`: 使用者
>>> * `assistant`: ChatGPT 的回應

>>基本上，傳過去的對話紀錄會長這個樣子。
>>>
    messages = [
        {"role": "system", "content":"ChatGPT的「人設」"},
        {"role": "user", "content": "使用者說的內容"},
        {"role": "assistant", "content": "ChatGPT回應內容"},
            ：
            ：
        {"role": "user", "content": prompt (最後說的)}
        ]

7 用Gradio打造網頁對話機器人介面
===

In [9]:
import gradio as gr

messages = [{"role":"system", "content":system}]

def mychatbot(prompt):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages = messages,
        model = model,
        max_tokens = 1024,
        temperature = 0.2,
        top_p = 0.95,
        )
    reply = chat_completion.choices[0].message.content
    return reply

iface = gr.Interface(
    mychatbot,
    inputs = "text",
    outputs = "text",
    title = title,
    description = descriptions)

iface.launch(share = True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://367772929f45653c9c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://367772929f45653c9c.gradio.live
